In [ ]:
from getpass import getpass
from itertools import product
import pathlib
import re
import warnings

import contextily as cx
from crossfire import Client
import geobr
import geopandas as gpd
import ipeadatapy
import libpysal.weights as pysal
import matplotlib.pyplot as plt
import numpy as np
import osmnx as ox
import pandas as pd
import pandana as pdna
import partridge as ptg
from scipy.stats import zscore
from sklearn.neighbors import KernelDensity
from shapely.geometry import LineString, Point
from shapely.ops import split
import seaborn as sns
from libpysal.io.fileio import FileIO
from tobler.area_weighted import area_interpolate

## Segurança Pública

In [ ]:
isp1 = pd.read_csv(
    '../database/6. Segurança Pública/Pedido_081_2023_part_1.csv',
    encoding='latin-1',
    sep=';',
    )
isp2 = pd.read_csv(
    '../database/6. Segurança Pública/Pedido_081_2023_part_2.csv',
    encoding='latin-1',
    sep=';',
    )

isp = pd.concat([isp1, isp2], ignore_index=True)

del isp1, isp2

In [ ]:
isp = (
    isp
    .loc[
        (isp.local == 'Via pública')
        & (isp.titulo_do != 'Encontro de ossada')
        #& (isp.ano == 2019)
        & ~(isp.titulo_do.str.contains('estupr', case=False))
        ]
    .assign(
        shape_id=lambda x: x.cisp.str.extract(r'(\d{3}).*').astype(int),
        titulo_do=lambda x: np.where(
            x.titulo_do.str.contains('roubo|extors|morte|homic', case=False),
            'violent_crime',
            x.titulo_do,
            )
        )
    .assign(
        titulo_do=lambda x: np.where(
            x.titulo_do.str.contains('furto', case=False),
            'non_violent_crime',
            x.titulo_do,
            )
         )
    )

In [ ]:
isp.head(3)

In [ ]:
crime_to_plot = (
    isp
    .groupby(
        ['ano', 'mes', 'shape_id', 'titulo_do']
        )
    .size()
    .round()
    .unstack('titulo_do')
    .groupby(level='shape_id')
    .median()
    .astype(int)
    .rename_axis('crime_nature', axis='columns')
    )

In [ ]:
crime_to_plot.head(3)

In [ ]:
cisp = gpd.read_file(
    '../database/6. Segurança Pública/CISPshp.zip'
    )

In [ ]:
rio = (
    geobr
    .read_metro_area()
    .pipe(lambda x: x.query("name_metro == 'RM Rio de Janeiro'"))
    .to_crs(31983)
    .buffer(-500).to_frame('rio')
    )

cisp = (
    cisp
    .to_crs(epsg=31983)
    .sjoin(rio)
    .drop_duplicates(subset='dp')
    .merge(crime_to_plot, left_on='dp', right_index=True)
    )

In [ ]:
cisp.explore('violent_crime', scheme='NaturalBreaks', cmap='RdYlBu_r')

In [ ]:
attractions = gpd.read_file('outputs/S100/total_attractions.gpkg')

flow_by_cisp = area_interpolate(
    source_df=attractions,
    target_df=cisp,
    extensive_variables=['fator_expansao']
    ).drop(columns='geometry')

cisp = (
    cisp
    .merge(flow_by_cisp, left_index=True, right_index=True)
    .assign(
        violent_crime=lambda x: x.violent_crime / x.fator_expansao * 100_000,
        non_violent_crime=lambda x: x.non_violent_crime / x.fator_expansao * 100_000,
        )
    )

In [ ]:
crime_by_hex = area_interpolate(
    source_df=cisp,
    target_df=db,
    intensive_variables=['violent_crime', 'non_violent_crime']
    ).drop(columns='geometry')

db = db.merge(crime_by_hex, left_index=True, right_index=True)

In [ ]:
db.explore('violent_crime', scheme='JenksCaspall', cmap='coolwarm')

## Fogo Cruzado

In [ ]:
shots = (
    gpd
    .read_file('outputs/S300/fogo_cruzado.gpkg')
    .assign(
        date=lambda db: pd.to_datetime(db.date)
        )
    .pipe(
        lambda x: x.loc[x.date < '2020-01-01']
        )
    )
    

### KDE

In [ ]:
model = KernelDensity(bandwidth=300, kernel='gaussian')
fit = model.fit(
    X=list(zip(shots.geometry.x, shots.geometry.y)),
    )

db = (
    db
    .assign(**{
        f'gunshot_kde': lambda df: np.exp(
            fit.score_samples(
                X=df[['X', 'Y']].to_numpy()
                )
            )
        })    
    )


In [ ]:
db.explore('gunshot_kde', cmap='inferno', scheme='headtailbreaks', prefer_canvas=True)